In [1]:
import random
import pandas as pd
import numpy as np
import os
import ydata_profiling
from sklearn.ensemble import RandomForestRegressor
from ydata_profiling import ProfileReport

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정